In [1]:
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
import pandas as pd
from datetime import date, timedelta
import requests
from bs4 import BeautifulSoup
import numpy as np
from datetime import datetime, time, date, timedelta
import datetime
from dateutil import parser


In [2]:
def feriados_anbima():
    """_summary_ busca lista de feriados padrão Brazil/ANBIMA


    Returns:
        dataframe: dataframe com lista de feriados até 2071
    """

    df = pd.read_excel(r'https://www.anbima.com.br/feriados/arqs/feriados_nacionais.xls')
    fer = list(df['Data'][0:len(df)-10])
    fer = [fer[i].date() for i in range(len(fer))]
    return fer

feriados = feriados_anbima()

In [3]:
def dfs_tabs(df_list, sheet_list, file_name):
    writer = pd.ExcelWriter(file_name,engine='xlsxwriter')   
    for dataframe, sheet in zip(df_list, sheet_list):
        dataframe.to_excel(writer, sheet_name=sheet, startrow=0 , startcol=0)   
    writer.save()

In [4]:
class b3_dados:
    def __init__(self, data_buscada):
        """
        Gera o objeto da b3 a partir de data_val em formato de datetime.date
        """
        self.data_buscada = data_buscada
        mes = self.data_buscada.month
        dia = self.data_buscada.day
        if self.data_buscada.month<10: mes='0'+str(self.data_buscada.month)
        if self.data_buscada.day<10: dia = '0'+str(self.data_buscada.day)
        self.dt_barra = f'{dia}/{mes}/{data_buscada.year}'
        self.dt_corrida = f'{data_buscada.year}{mes}{dia}'
        self.headers = {"User-Agent":'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.100 Safari/537.36'}
    
    def _baixar_pre(self):

        link = f'https://www2.bmf.com.br/pages/portal/bmfbovespa/boletim1/TxRef1.asp?Data={self.dt_barra}&Data1={self.dt_corrida}&slcTaxa=PRE'
        page = requests.get(link, headers=self.headers, verify=False)
        soup = BeautifulSoup(page.content, 'html.parser')
        texto = soup.find_all('td')
        dias, taxas252, taxas360 = [], [], []
        tables = ["['tabelaConteudo1']", "['tabelaConteudo2']"]
        for i in range(0,len(texto),3):
            try:
                if str(texto[i]['class']) in tables:
                    if i<=len(texto)-2:
                        dias.append(int(texto[i].text.replace('\r\n','').replace(',','.').replace(' ','')))
                        taxas252.append(float(texto[i+1].text.replace('\r\n','').replace(',','.').replace(' ',''))/100)
                        taxas360.append(float(texto[i+2].text.replace('\r\n','').replace(',','.').replace(' ',''))/100)
            
            except:
                pass
        return pd.DataFrame({'taxas252':taxas252,'taxas360':taxas360}, index=dias)   
    
    
    
    def _baixa_cupom(self):
        """
        Dado a data_buscada baixa a curva de cupom limpo
        """
        link = f'https://www2.bmf.com.br/pages/portal/bmfbovespa/boletim1/TxRef1.asp?Data={self.dt_barra}&Data1={self.dt_corrida}&slcTaxa=DIC'
        page = requests.get(link, headers=self.headers, verify=False)
        soup = BeautifulSoup(page.content, 'html.parser')
        texto = soup.find_all('td')
        dias, taxas = [], []
        tabelas = ["['tabelaConteudo1']", "['tabelaConteudo2']"]
        for i in range(len(texto)):
            try:
                if str(texto[i]['class']) in tabelas:
                    tratado = texto[i].text.replace('\r\n','').replace(',','.').replace(' ','')
                    if i==0 or i%2==0:
                        dias.append(int(tratado))
                    else:
                        taxas.append(float(tratado)/100)
            except:
                pass
        return pd.DataFrame(data=taxas, index=dias, columns={'taxas252'})
    

In [5]:
def generate(dias_dif):
    'Inserir dias_dif pela qtd de dias no passado você quer a tabela'
    if isinstance(dias_dif,int) == False:
        return "Inserir valor númerico inteiro"
    else:
        data_busca = date.today() - timedelta(dias_dif)
        #print(data_busca)
        while data_busca.weekday() in (5,6) or data_busca in feriados:
            data_busca = data_busca - timedelta(days=1)
            #print(f'nova_data_busca{data_busca}')
        ano = data_busca.year
        mes = data_busca.month
        dia = data_busca.day
    
        dado_di = b3_dados(date(ano,mes,dia))._baixar_pre()
        dado_di.drop('taxas360', axis=1, inplace=True)
        dado_cupom= b3_dados(date(ano,mes,dia))._baixa_cupom()
        
      #Criar Interpolação Cupom IPCA
        datas = []
        last_element = int(dado_cupom.iloc[-1].name)
        if last_element is None: print('vazio')
        for i in range(0,last_element+1):
            datas.append(i)
        datas = pd.DataFrame(datas)  
        df2 = pd.concat([datas, dado_cupom], axis=1)
        df2.set_index(df2.columns[0])
        ser1 = df2['taxas252']
        ser1 = ser1.interpolate(method='polynomial', order=2)
        ipca_tabela = ser1.to_frame()
        ipca_tabela.dropna(inplace=True)
        ipca_tabela = ipca_tabela.head(36000)
        ipca_tabela = ipca_tabela.rename(columns={"taxas252":"CUPOM_IPCA"})
        dado_cupom = ipca_tabela
        
      #Criar Interpolação DI
        datas = []
        last_element = int(dado_di.iloc[-1].name)
        for i in range(0,last_element+1):
            datas.append(i)
        datas = pd.DataFrame(datas)  

        df2 = pd.concat([datas, dado_di], axis=1)
        df2.set_index(df2.columns[0])
        ser1 = df2['taxas252']
        ser1 = ser1.interpolate(method='polynomial', order=2)
        dif = ser1.to_frame()
        dif.dropna(inplace=True)
        dif = dif.head(36000)
        dif = dif.rename(columns={"taxas252":"DI-1"})
        dado_di = dif
        
       #Criar_Implicita:
        temp_ipca1 = pd.merge(dado_di, dado_cupom, left_index=True, right_index=True)
        temp_ipca1['Implicita'] = ((1+temp_ipca1['DI-1'])/(1+temp_ipca1['CUPOM_IPCA']))-1
        implicita = temp_ipca1['Implicita']
        
    return dado_di,dado_cupom,implicita

In [6]:
def merge_dfs():
    """Realiza o merge dos datasframes para cada um dos indexadores.

    Returns:
        dataframe: retorna 3 dataframes[DI,Cupom_IPCA,Implicita] - referente aos prazos de: DU=-1 , DU=-2, DU=-5, DU=-21
    """
    dfs_di = [di_1,di_2,di_5,di_21]
    result_di = pd.concat(dfs_di, join='inner', axis=1).fillna(np.nan)
    result_di.columns = ['DI-D1', 'DI-D2', 'DI-D5', 'DI-D21']
    result_di = result_di.iloc[45:]

    dfs_cupom_ipca = [cupom_1,cupom_2,cupom_5,cupom_21]
    result_cupom_ipca = pd.concat(dfs_cupom_ipca, join='inner', axis=1).fillna(np.nan)
    result_cupom_ipca.columns = ['Cupom_IPCA-D1', 'Cupom_IPCA-D2', 'Cupom_IPCA-D5', 'Cupom_IPCA-D21']
    result_cupom_ipca = result_cupom_ipca.iloc[45:]

    dfs_implicita = [implicita_1,implicita_2,implicita_5,implicita_21]
    result_implicita = pd.concat(dfs_implicita, join='inner', axis=1).fillna(np.nan)
    result_implicita.columns = ['Implicita-D1', 'Implicita-D2', 'Implicita-D5', 'Implicita-D21']
    result_implicita = result_implicita.iloc[45:]

    return result_di,result_cupom_ipca,result_implicita



In [7]:
def criar_tabela_anos():
    """Realiza o filtros dos datasframes para cada um dos indexadores pelas datas
       DI = 360,720,1080,1440,1800,3600 dias corridos
       IPCA/Inflação Implicita: Vencimentos de 1,2,3,4,6,8,13,28,33 anos que faz referência aos vencimentos de
       NTN-Bs.
    Returns:
        dataframe: retorna 3 dataframes[DI,Cupom_IPCA,Implicita] com os prazos supracitados.
    """
    DI_tabela_anos = DI.filter(items = [360,720,1080,1440,1800,3600], axis=0)
    DI_tabela_anos = DI_tabela_anos.rename(index={360: "1 Ano", 720: "2 Anos", 1080: "3 anos", 1440: "4 Anos", 1800: "5 Anos", 3600: "10 Anos"})
    
    cupom_ipca_tabela_anos = cupom_ipca.filter(items = [365,
                               730,
                               1096,
                               1461,
                               2191,
                               2922,
                               4748,
                               10220,
                               12045], axis=0)

    cupom_ipca_tabela_anos = cupom_ipca_tabela_anos.rename(index=
                                {365: "1 Ano", 
                                730: "2 Anos",
                                1096: "3 anos",
                                1461: "4 Anos",
                                2191: "6 Anos",
                                2922: '8 Anos',
                                4748: "13 Anos", 
                                10220: "28 anos",
                                12045 : "33 Anos"
                                })
    
    implicita_tabela_anos = implicita.filter(items = [365,
                               730,
                               1096,
                               1461,
                               2191,
                               2922,
                               4748,
                               10220,
                               12045], axis=0)

    implicita_tabela_anos = implicita_tabela_anos.rename(index=
                                {365: "1 Ano", 
                                730: "2 Anos",
                                1096: "3 anos",
                                1461: "4 Anos",
                                2191: "6 Anos",
                                2922: '8 Anos',
                                4748: "13 Anos", 
                                10220: "28 anos",
                                12045 : "33 Anos"
                                })
    
    return DI_tabela_anos,cupom_ipca_tabela_anos,implicita_tabela_anos



In [8]:
print("*"*50)
print('Iniciando')
print("*"*50)
generate_1 = generate(1)
di_1 = generate_1[0]
cupom_1 = generate_1[1]
implicita_1 = generate_1[2]

generate_2 = generate(2) 
di_2 = generate_2[0]
cupom_2 = generate_2[1]
implicita_2 = generate_2[2]

generate_5 = generate(5)
di_5 = generate_5[0]
cupom_5 = generate_5[1]
implicita_5 = generate_5[2]

generate_21 = generate(21)
di_21 = generate_21[0]
cupom_21 = generate_21[1]
implicita_21 = generate_21[2]


print('Criando os Merges')
print("*"*50)
DI = merge_dfs()[0]
cupom_ipca = merge_dfs()[1]
implicita = merge_dfs()[2]

hoje = date.today()
DI["dataref"] = hoje
cupom_ipca["dataref"] = hoje
implicita["dataref"] = hoje


print('Criando tabelas')
print("*"*50)
DI_tabela_anos = criar_tabela_anos()[0]
DI_tabela_anos['d'] = date.today()
cupom_ipca_tabela_anos = criar_tabela_anos()[1]
implicita_tabela_anos = criar_tabela_anos()[2]

print('Gerando Excel')
print("*"*50)
dfs = [DI, cupom_ipca,implicita,DI_tabela_anos,cupom_ipca_tabela_anos,implicita_tabela_anos]
sheets = ['curva_pre','cupom_ipca','inflacao_implicita','DI_tabela_anos','cupom_ipca_tabela_anos','implicita_tabela_anos']    
dfs_tabs(dfs, sheets, 'dados_economicos190123_.xlsx')
print("Finalizado")

**************************************************
Iniciando
**************************************************
Criando os Merges
**************************************************
Criando tabelas
**************************************************
Gerando Excel
**************************************************
Finalizado
